# Projet clustering

Objectif du projet : Utiliser le corpus export_articles_EGC_2004_2018 et faire le bilan de l'évolution de la communauté EGC ces 20 dernières années et tenter d'en prédire l'avenir.

##  Notes :
J'ai abandonné l'utilisation de TreeTagger pour lemmatiser : j'ai pas réussi à le faire fonctionner.

Ainsi que Gensim : la forme de ses corpus est étrange et je pense que c'est une perte de temps à se conformer à leur modèle.

### Imports

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.stem.porter import *
from nltk.corpus import stopwords
from random import randint

### Ouverture du fichier de données

# Fichier de données original.
doc = pd.read_csv("export_articles_EGC_2004_2018.csv", sep='\t')
doc

In [ ]:
# Fichier de données corrigé.
# La correction va jusqu'à la ligne 66.
doc = pd.read_csv("export_articles_EGC_2004_2018_Copie.csv", sep='\t')
doc

En travaillant avec title et abstract, on va réaliser les transformations.
Tous les documents ont des titres mais ils n'ont pas tous d'abstract.
On va donc utiliser une concaténation du titre et de l'abstract afin d'être sûr d'avoir de l'information.

### Lemmatisation et nettoyage des sujets

Il faut mettre les textes en minuscules, tokeniser, retirer la ponctuation, retirer les stopwords puis lemmatiser.

In [ ]:
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        if item not in stopwords.words("french"):
            stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = text.translate(str.maketrans('','',string.punctuation))
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

Attention : l'étape suivante est longue.

In [ ]:
resume = [tokenize(doc["title"][0].lower())]
abstract = [tokenize(str(doc["abstract"][0].lower()))]
for line in range(len(doc)): # title et abstract ont la même taille.
    if line != 0:
        resume.append(tokenize(doc["title"][line].lower()))
        abstract.append(tokenize(str(doc["abstract"][line]).lower()))

Le traitment des données est acceptable. Un meilleur traitement pourrait être envisagé.

Il faut obtenir un seul jeu de données avec les titres et les abstracts pour pouvoir les traiter et éliminer les cas qui n'ont pas d'abstract.

Il est possible que seul le titre suffise. Il est possible que la racinisation ne soit pas nécéssaire.

In [ ]:
for line in range(len(abstract)):
    if abstract[line] != "nan":
        for element in abstract[line]:
            resume[line].append(element)

### Tf-Idf

Attention : l'étape suivante est longue.

In [ ]:
dico = []
for line in resume:
    for word in line:
        dico.append(word)
for word in dico:
    if dico.count(word) != 1:
        dico.remove
len(dico)
# 93849 (2 fois)

Il faudrait s'assurer que dico ne contienne que des mots distincts. Pour réduire sa taille.

In [ ]:
# Calcul du Tf.
tf = []
for x in range(len(resume)):
    tf.append([])
    for y in range(len(resume[x])):
        tf[x].append(resume[x].count(resume[x][y]))

Attention : l'étape suivante est longue.

In [ ]:
# Calcul du Df.
df = {}
for x in range(len(dico)):
    sum = 0
    for y in range(len(resume)):
        if dico[x] in resume[y]:
            sum = sum + 1
    df[dico[x]] = sum

In [ ]:
# Calcul de l'Idf
idf = {}
for word in df:
    idf[word] = np.log10(len(resume)/df[word])

In [ ]:
# Calcul du Tf-Idf.
tfidf = []
for x in range(len(resume)):
    tfidf.append({})
    for y in range(len(resume[x])):
        tfidf[x][resume[x][y]] = tf[x][y] * idf[resume[x][y]]
len(tfidf)

In [ ]:
tfidf[1]

Il faut trier les Tf-Idf par valeur et récupérer les 5 meilleurs.

### Calcul du LSA

Le LSA va calculer, pour un document donné, les documents les plus similaires à son sujet. On classe le résultat le plus similaire dans le cluster du document donné. On reproduit cette étape et on arrête quand à un nombre de clusters ou une similarité trop faible.

En gros, le LSA nous fournit une similarité qu'on utilise pour former une classification hiérarchique.

On aura donc des clusters par sujets. On pourra élaborer nos techniques de datation pour déterminer les tendances et, si possible, créer de nouveaux clusters (avec les K-moyennes) plus précis.

### Clustering Hiérarchique Ascendant (HAC)

Le vrai point d'intérêt de cet algorithme est le seuil de similarité qui servira de point d'arrêt. Pour l'instant, je ne sais pas quel sera ce seuil, il faudra attendre les résultats du clustering.

In [ ]:
# Je suppose qu'on obtient une liste de listes (liste de tous les clusters individuels) à la fin du clustering.
# On pourra modifier l'algorithme ou les données pour que cela fonctionne
simi = 1 # Je crois que la similarité va de 0 à 1, à vérifier. # Similarité du dernier cluster retenu.
seuil = # A voir
num = 0 # Numéro du cluster actuellement étudié.
qt = len(# Liste clusters) # Quantité de clusters, évolue au cours de l'algorithme.
print(qt) # Nombre de clusters avant HAC.
while similarity >= seuil:
    num = randint(0,qt)
    # Calcul de la similarité entre le cluster[num] et les autres
    # Je suppose que la similarité renvoit un dictionnaire (cluster, similarité). A modifier sans doute.
    simi = list(res.values())[0]
    cluster[num].append(list(res.keys())[0])
    # A voir si la liste de cluster évolue toute seule ou s'il faut manuellement retirer le cluster qu'on vient d'affecter.
print(qt) # Nombre de clusters après HAC.

### Évaluation de la classification

On va maximiser la distance inter cluster et minimiser la distance intra cluster

Distance intra max < Distance inter min

Comparons l'efficacité de la similarité de Jaccard et de la similarité Cosine

In [ ]:
def jaccard(vec1, vec):
    commun = 0
    diff = 0
    for key in vec1.keys():
        if key in vec.keys():
            commun = commun + 1
        else:
            diff = diff + 1
    if diff == 0:
        return -1
    return commun/diff

def cosine(vec1, vec2):
    v1 = np.array(list(vec1.values()))
    v2 = np.array(list(vec2.values()))
    # La partie qui suit devra être retirée plus tard quand on aura des clusters normalisés.
    if len(vec1) > len(vec2):
        v1 = v1[:len(vec2)]
    else:
        v2 = v2[:len(vec1)]
    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))  

Les exemples qui suivent sont basés sur les résultats du Tf-Idf. La véritable utilisation se fera entre deux clusters.

In [ ]:
print("Distance entre tfidf[1] et tfidf[2]")
print("Jaccard :",jaccard(tfidf[1],tfidf[2]))
print("Cosine :",cosine(tfidf[1],tfidf[2]))

In [ ]:
print("Distance entre tfidf[1] et tfidf[3]")
print("Jaccard :",jaccard(tfidf[1],tfidf[3]))
print("Cosine :",cosine(tfidf[1],tfidf[3]))

In [ ]:
print("Distance entre tfidf[2] et tfidf[3]")
print("Jaccard :",jaccard(tfidf[2],tfidf[3]))
print("Cosine :",cosine(tfidf[2],tfidf[3]))

In [ ]:
print("Distance entre tfidf[1] et tfidf[1]")
print("Jaccard :",jaccard(tfidf[1],tfidf[1]))
print("Cosine :",cosine(tfidf[1],tfidf[1]))

https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50 nous informe que Jaccard est plus intéréssant si la répétition des mots dans le texte n'est pas important (notre cas). A voir plus tard lequel des deux on garde (ou les deux si possible).